<a href="https://colab.research.google.com/github/TatyanaLanceva/Netology/blob/main/4_1_Functions_and_working_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

    оценка 2 и ниже — низкий рейтинг;
    оценка 4 и ниже — средний рейтинг;
    оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [1]:
# Загружаем библиотеки
import pandas as pd
import json
import csv
import re
from scipy.stats import pearsonr

In [ ]:
# Чтение CSV-файла movies
df_movies = pd.read_csv('movies.csv')
# Просмотр первых пяти строк таблицы
print(df_movies.head())
print()
df_movies.info()

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [ ]:
%%time
# Чтение CSV-файла ratings
df_ratings = pd.read_csv('ratings.csv')
# Просмотр первых пяти строк таблицы
print(df_ratings.head())
print()
df_ratings.info()

   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
CPU times: user 71.9 ms, sys: 14.8 ms, total: 86.7 ms
Wall time: 104 ms


In [ ]:
# Объединим данные для получения общей информации
# Слияние DataFrame
joined = df_ratings.merge(df_movies, left_on='movieId', right_on='movieId', how='left')
# Краткий просмотр
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [ ]:
# Соберем средний рейтин для каждого фильма

joined.groupby('title').mean('rating').round(1).head(-5)

,userId,movieId,rating,timestamp
title,,,,
"""Great Performances"" Cats (1998)",382.0,51372.0,1.8,1.272201e+09
$9.99 (2008),362.7,74486.0,3.8,1.296821e+09
'Hellboy': The Seeds of Creation (2004),134.0,97757.0,2.0,1.361244e+09
'Neath the Arizona Skies (1934),207.0,70121.0,0.5,1.258440e+09
'Round Midnight (1986),377.0,26564.0,2.2,1.215959e+09
...,...,...,...,...
Zulu (1964),376.5,5899.0,4.0,1.137965e+09
Zulu (2013),624.0,116207.0,1.5,1.461357e+09
[REC] (2007),155.7,57274.0,3.3,1.377138e+09


In [ ]:
# Добавим функцию с классификацией рейтинга.

def get_class(rating):

    if rating <= 2:
        return 'низкий рейтинг'
    elif rating <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

# Добавляем новый столбец 'class' и запишем результат классификации

joined['class'] = joined['rating'].apply(get_class)

# Выводим результат
joined.head(-5)

,userId,movieId,rating,timestamp,title,genres,class
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,средний рейтинг
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,средний рейтинг
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,средний рейтинг
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,низкий рейтинг
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,средний рейтинг
...,...,...,...,...,...,...,...
99994,671,5952,5.0,1063502716,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,высокий рейтинг
99995,671,5989,4.0,1064890625,Catch Me If You Can (2002),Crime|Drama,средний рейтинг
99996,671,5991,4.5,1064245387,Chicago (2002),Comedy|Crime|Drama|Musical,высокий рейтинг
99997,671,5995,4.0,1066793014,"Pianist, The (2002)",Drama|War,средний рейтинг


# Задание 2

Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

12345
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.


In [ ]:
# Чтение CSV-файла movies
df_keywords = pd.read_csv('keywords.csv')
# Просмотр первых пяти строк таблицы
print(df_keywords.head(-5))
print()
df_keywords.info()



                          keyword     shows
0                              вк  64292779
1                   одноклассники  63810309
2                           порно  41747114
3                            ютуб  39995567
4                       вконтакте  21014195
...                           ...       ...
99990           сколько лет обаме      3799
99991                сергей жорин      4056
99992    остаться в живых 5 сезон      3696
99993  петростат официальный сайт      3634
99994      мк газета свежий номер      3750

[99995 rows x 2 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   keyword  100000 non-null  object
 1   shows    100000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [ ]:
# Перевод geo_data в удобный для поиска формат
regions = {}
for region, cities in geo_data.items():
    for city in cities:
        regions[city.lower()] = region

# Сделаем функцию для классификации региона
def classify_region(keyword):
    keyword_lower = keyword.lower()
    for city, region in regions.items():
        if city in keyword_lower:
            return region
    return 'undefined'

# Применим функцию к каждому запросу:
df_keywords['region'] = df_keywords['keyword'].apply(classify_region)
df_keywords.head(-5)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
...,...,...,...
99990,сколько лет обаме,3799,undefined
99991,сергей жорин,4056,undefined
99992,остаться в живых 5 сезон,3696,undefined
99993,петростат официальный сайт,3634,undefined


In [ ]:
#  При кратком выводе мы не видим в столбце region ничего, кроме undefined, поэтому стоит
# сделать группировку по столбцу region и проверить другие значения, сгруппируем их, видно, что замена произошла.

df_keywords.groupby('region').count().head()

,keyword,shows
region,,
undefined,99253,99253
Дальний Восток,94,94
Северо-Запад,266,266
Центр,387,387


# Задание 3.

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010 года.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:
        для каждой строки пройдите по всем годам списка years;
        если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.


In [ ]:
# В первом задании брались данные из MovieLens 25M Dataset, заберем оттуда объединенные данные


joined.head()

,userId,movieId,rating,timestamp,title,genres,class
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,средний рейтинг
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,средний рейтинг
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,средний рейтинг
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,низкий рейтинг
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,средний рейтинг


In [ ]:
# В переменную years запишем список из всех годов с 1950 по 2010 года, пишем до 2011 год, потому что крайний год диапазона не включается.

years = list(range(1950, 2011))


In [ ]:
%%time
# Напишем функцию production_year, которая каждой строке из названия фильма выставляет год выпуска, а если не находит год, то ставит 1900.

def production_year(movie_title):
    for year in years:
        if str(year) in movie_title:
            return int(year)
    return 1900

joined['year'] = joined['title'].apply(production_year)

joined.head()

CPU times: user 1.5 s, sys: 3.64 ms, total: 1.5 s
Wall time: 1.57 s


,userId,movieId,rating,timestamp,title,genres,class,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,средний рейтинг,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,средний рейтинг,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,средний рейтинг,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,низкий рейтинг,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,средний рейтинг,1989


In [ ]:
# Проверим группировкой годов, что все по годам заполнено по заданию

joined.groupby('year').count().head(70)

,userId,movieId,rating,timestamp,title,genres,class
year,,,,,,,
1900,6925,6925,6925,6925,6925,6925,6925
1950,236,236,236,236,236,236,236
1951,243,243,243,243,243,243,243
1952,152,152,152,152,152,152,152
1953,218,218,218,218,218,218,218
...,...,...,...,...,...,...,...
2006,1786,1786,1786,1786,1786,1786,1786
2007,1607,1607,1607,1607,1607,1607,1607
2008,1544,1544,1544,1544,1544,1544,1544


In [ ]:
# Посчитем средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируем результат по убыванию рейтинга.

joined.groupby('year')['rating'].mean().round(1).sort_values(ascending=False).head(100)

,rating
year,
1951,4.0
1952,4.0
1974,4.0
1954,4.0
1957,4.0
...,...
1996,3.4
2000,3.4
2003,3.4


In [ ]:
corr, p_value = pearsonr(joined['year'], joined['rating'])
print(f'Коэффициент корреляции Пирсона: {corr:.2f}, p-значение: {p_value:.10f}.')

Коэффициент корреляции Пирсона: -0.06, p-значение: 0.0000000000.


**Выводы**: Коэффициент корреляции Пирсона - 0,06 показывает очень слабую отрицательную корелляцию, можно предположить, что с увеличением года выпуска рейтинг снижается, но эта зависимость слишком мала, чтобы считаться значимой. p-значение стремится к нулю, что показывает статистическую значимость, но в сочетании с величиной коэффициента корреляции Пирсона также можно пренебречь.